In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense

C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K
from keras.engine.topology import Layer

import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=np.nan)

sess = tf.Session()


class Hebbian(Layer):
    
    
    def __init__(self, output_dim, lmbda=1.0, eta=0.0005, connectivity='random', connectivity_prob=0.25, **kwargs):
        '''
        Constructor for the Hebbian learning layer.

        args:
            output_dim - The shape of the output / activations computed by the layer.
            lambda - A floating-point valued parameter governing the strength of the Hebbian learning activation.
            eta - A floating-point valued parameter governing the Hebbian learning rate.
            connectivity - A string which determines the way in which the neurons in this layer are connected to
                the neurons in the previous layer.
        '''
        self.output_dim = output_dim
        self.lmbda = lmbda
        self.eta = eta
        self.connectivity = connectivity
        self.connectivity_prob = connectivity_prob

        super(Hebbian, self).__init__(**kwargs)
        
    
    
    def random_conn_init(self, shape, dtype=None):
        A = np.random.normal(0, 1, shape)
        A[self.B] = 0
        return tf.constant(A, dtype=tf.float32)


    def zero_init(self, shape, dtype=None):
        return np.zeros(shape)


    def build(self, input_shape):
        # create weight variable for this layer according to user-specified initialization
        if self.connectivity == 'random':
            self.B = np.random.random(input_shape[0]) < self.connectivity_prob
        elif self.connectivity == 'zero':
            self.B = np.zeros(self.output_dim)
            
        if self.connectivity == 'all':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer='uniform', trainable=False)
        elif self.connectivity == 'random':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.random_conn_init, trainable=False)
        elif self.connectivity == 'zero':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.zero_init, trainable=False)
        else:
            raise NotImplementedError


        # call superclass "build" function
        super(Hebbian, self).build(input_shape)


    def call(self, x):
        x_shape = tf.shape(x)
        batch_size = tf.shape(x)[0]

        # reshape to (batch_size, product of other dimensions) shape
        x = tf.reshape(x, (tf.reduce_prod(x_shape[1:]), batch_size))

        # compute activations using Hebbian-like update rule
        activations = x + self.lmbda * tf.matmul(self.kernel, x)  # why "x +", should this be removed?

        # compute outer product of activations matrix with itself
        outer_product = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(x, 0))  # Why outer product? Should this line be removed?

        # update the weight matrix of this layer
        self.kernel = self.kernel + tf.multiply(self.eta, tf.reduce_mean(outer_product, axis=2)) # Still why outer prod?
        self.kernel = tf.multiply(self.kernel, self.B) # zeroing node connections that are meant to be zeros
        return K.reshape(activations, x_shape)

        
        


In [3]:
model = Sequential()
model.add(Hebbian(input_shape = (256,1), output_dim = 24))

In [4]:
model.compile(loss='mse', optimizer='adam')